In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os
root_depth = 2
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from spykshrk.realtime.decoder_process import PointProcessDecoder

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [4]:
# Setup

delta = 1

start_x = 0
end_x = 100
num_bins = 100

pp_decoder = PointProcessDecoder(pos_range=[start_x, end_x],
                                 pos_bins=num_bins,
                                 time_bin_size=delta,
                                 arm_coor=[[start_x, end_x]])

pp_decoder.select_ntrodes([1, 2])


In [5]:
# spike observation distributions
x_bins = np.linspace(start_x, end_x - end_x/num_bins, num_bins)
spk1 = gaussian(x_bins, 30, 10)
spk1 = spk1/(spk1.sum() * num_bins / (end_x - start_x))
spk2 = gaussian(x_bins, 40, 10)
spk2 = spk2/(spk2.sum() * num_bins / (end_x - start_x))
spk3 = gaussian(x_bins, 60, 10)
spk3 = spk3/(spk3.sum() * num_bins / (end_x - start_x))
spk4 = gaussian(x_bins, 70, 10)
spk4 = spk4/(spk4.sum() * num_bins / (end_x - start_x))


In [6]:
# Add spikes to model and increment bin
pp_decoder.update_position(1, 1)
pp_decoder.add_observation(1, spk1)
pp_decoder.add_observation(1, spk2)

posterior1 = pp_decoder.increment_bin()
posterior2 = pp_decoder.increment_no_spike_bin()

pp_decoder.add_observation(1, spk3)
pp_decoder.add_observation(1, spk4)

observation3 = pp_decoder.observation

posterior3 = pp_decoder.increment_bin()

In [7]:
plt.plot(x_bins, posterior1, 'r')
plt.plot(x_bins, posterior2, 'c')
plt.plot(x_bins, observation3/10, 'b')
plt.plot(x_bins, posterior3, 'b')
plt.show()

In [8]:
plt.imshow(pp_decoder.transition_mat)
plt.show()